In [1]:
!pip install numpy scipy librosa unidecode inflect librosa
!apt-get update
!apt-get install -y libsndfile1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: soundfile
    Found existing installation: soundfile 0.11.0
    Uninstalling soundfile-0.11.0:
      Successfully uninstalled soundfile-0.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wfdb 4.1.0 requires SoundFile<0.12.0,>=0.10.0, but you have soundfile 0.12.1 which is incompatible.
Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5002 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]       
Get:3 https://packages.cloud.google.com/apt google-fast-socket InRelease [5015 B]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRele

In [2]:
from scipy.io.wavfile import write
import torch
from IPython.display import Audio
import nltk
from nltk.corpus import brown
import os
from tqdm import tqdm
import pandas as pd
import shutil
import json
import gc
import re
from IPython.display import FileLink

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')

/opt/conda/lib/python3.7/site-packages/torch/hub.py:268: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:14: UserWarning: p

In [5]:
tacotron2.decoder.max_decoder_steps = 3000

In [6]:
waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()
print('done')

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


done


In [7]:
def tts(text, rate=16000, write_to_file=False, path='audio.wav', return_auido=True):
    sequences, lengths = utils.prepare_input_sequence([text])
    with torch.no_grad():
        mel, _, _ = tacotron2.infer(sequences, lengths)
        audio = waveglow.infer(mel)
        audio_numpy = audio[0].data.cpu().numpy()
    if write_to_file:
        write(path, rate, audio_numpy)
    if return_auido:
        return Audio(audio_numpy, rate=rate)

In [58]:
# def tts2(text, rate=16000, write_to_file=False, path='audio.wav', return_auido=True, threshold=20):
#     sequences, lengths = utils.prepare_input_sequence([text])
#     with torch.no_grad():
#         mel, _, _ = tacotron2.infer(sequences, lengths)
#         audio = waveglow.infer(mel)
#         audio_numpy = audio[0].data.cpu().numpy()
#     if len(audio_numpy) // rate > threshold:
#         print(f'not accepted because length is {len(audio_numpy) // rate} for path {path}')
#         return False
#     if write_to_file:
#         write(path, rate, audio_numpy)
#         return True
#     if return_auido:
#         return Audio(audio_numpy, rate=rate)

In [ ]:
# os.makedirs('dataset', exist_ok=True)

In [ ]:
# PATH = 'audio.wav'
# tts("jirrrrrrr jirrrrakkkk.", rate=22050)

In [ ]:
# os.remove(PATH)

In [8]:
# nltk.download('brown')

# sentences = brown.sents()
# len(sentences)

[nltk_data] Downloading package brown to /usr/share/nltk_data...
[nltk_data]   Package brown is already up-to-date!


57340

In [12]:
# strings = ["hello world", "hi", "how are you doing"]
# strings.sort(key=lambda x: len(x.split()))
# print(strings)

['hi', 'hello world', 'how are you doing']


In [13]:
# MAX_LENGTH_ACCEPTABLE = 47
# max_length = 0
# max_sentence = None
# c = 0
# for item in sentences:
#     if len(item) > MAX_LENGTH_ACCEPTABLE:
#         c += 1
#     if MAX_LENGTH_ACCEPTABLE + 1 > len(item) > max_length :
#         max_length = len(item)
#         max_sentence = item
# print(len(max_sentence))
# print(len(sentences) - c)

47
55173


In [ ]:
# print(' '.join(max_sentence))
# tts(' '.join(max_sentence), rate=22050)

In [14]:
# accepted_sentences = []
# unaccepted_sentences = []
# for item in sentences:
#     if len(item) <= MAX_LENGTH_ACCEPTABLE:
#         accepted_sentences.append(item)
#     else:
#         unaccepted_sentences.append(item)
# print(len(accepted_sentences))

55173


In [15]:
# unaccepted_sentences.sort(key=lambda x: len(x))
# len(unaccepted_sentences[0])

48

In [25]:
# s = ' '.join(unaccepted_sentences[25])
# print(len(unaccepted_sentences[25]))
# print(s)
# a, b = tts(s, rate=22050)

48
The most surprising thing about the Twenty-second Congress of the Soviet Communist Party is that it is surprising -- perhaps quite as much , in its own way , as the Twentieth Congress of 1956 , which ended with that famous `` secret '' report on Stalin .


In [27]:
# len(b) // 22050

16

In [ ]:
# size = len(accepted_sentences) // 10 + 3 
# sublists = [accepted_sentences[i:i + size] for i in range(0, len(accepted_sentences), size)]
# print([len(sublists[i]) for i in range(len(sublists))])

In [ ]:
# meta_data = []
# localized_meta_data = {}
# BASIC_PATH = 'dataset'
# c = 0
# for i, sublist in enumerate(sublists):
#     part_id = i + 1
#     BASIC_PATH_PART = f'{BASIC_PATH}/part{part_id}/audios'
#     local_data = []
#     for item in sublist:
#         c += 1
#         sentence = ' '.join(item)
#         sentence = re.sub(r'\s+([?.!"])', r'\1', sentence)
#         path = f'{BASIC_PATH_PART}/audio_{c}.wav'
#         local_data.append({'id':c,'file_path': path, 'text': sentence})
#         meta_data.append({'id':c,'file_path': path, 'text': sentence})
#     localized_meta_data[f'part{part_id}'] = local_data

In [ ]:
# meta_data[11040:11046]

In [ ]:
# localized_meta_data['part3'][:5]

In [8]:
BASIC_PATH = 'dataset'
META_DATA_LOCALIZED_PATH = f'{BASIC_PATH}/localized_metadata.json'
with open(META_DATA_LOCALIZED_PATH, 'r') as f:
    localized_meta_data = json.load(f)

In [40]:
# META_DATA_GLOBAL_PATH = f'{BASIC_PATH}/global_metadata.json'
# with open(META_DATA_GLOBAL_PATH, 'r') as f:
#     global_meta_data = json.load(f)

In [ ]:
# with open(f'{BASIC_PATH}/global_metadata.json', 'w') as f:
#     json.dump(meta_data, f)
# META_DATA_LOCALIZED_PATH = f'{BASIC_PATH}/localized_metadata.json'
# with open(META_DATA_LOCALIZED_PATH, 'w') as f:
#     json.dump(localized_meta_data, f)

In [ ]:
# shutil.rmtree(BASIC_PATH)
# os.makedirs(BASIC_PATH, exist_ok=True)

In [ ]:
# shutil.rmtree(f'{BASIC_PATH}/part1')

In [9]:
os.listdir(BASIC_PATH)

['part10',
 'part8',
 'global_metadata.json',
 'part6',
 'part7',
 'localized_metadata.json',
 'part9']

In [10]:
def create_audios(part_id, meta_data_localized_path, start_index=None):
    with open(meta_data_localized_path, 'r') as f:
        meta_data = json.load(f)
    meta_data = meta_data[f'part{part_id}']
    
    os.makedirs(f'{BASIC_PATH}/part{part_id}', exist_ok=True)
    BASIC_PATH_PART = f'{BASIC_PATH}/part{part_id}/audios'
    os.makedirs(BASIC_PATH_PART, exist_ok=True)
    
    first_index = meta_data[0]['id']
    if start_index is None:
        start_index = first_index
    
    for i in tqdm(range(start_index - first_index, len(meta_data))):
        tts(meta_data[i]['text'], rate=22050, write_to_file=True, path=meta_data[i]['file_path'], return_auido=False)
        
        if i % 20 == 0:
            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()

In [ ]:
# create_audios(1, META_DATA_LOCALIZED_PATH, start_index=133)

In [ ]:
# shutil.make_archive('dataset_part1', 'zip', f'{BASIC_PATH}/part1')

In [ ]:
# import tarfile
# with tarfile.open('dataset_part1.tar.gz', 'w:gz') as tar:
#     tar.add(f'{BASIC_PATH}/part1', arcname='dataset_part1')

In [ ]:
# os.remove('dataset_part1.tar.gz')

In [ ]:
# FileLink(r'dataset_part1.zip')

In [ ]:
# create_audios(2, META_DATA_LOCALIZED_PATH, start_index=8641)

In [ ]:
# bad ids:
# 9710 part2
# 19729 part4
# 20309 part4
# 26563 part5
# 27331 part5
# 27334 part5
# 27377 part5
# 27679 part6

In [49]:
# bad_ids = [(9710, 2), (19729, 4), (20309, 4), (26563, 5), (27331, 5), (27334, 5), (27377, 5), (27679, 6)]

In [62]:
# global_meta_data[9710 - 1]

{'id': 9710,
 'file_path': 'dataset/part2/audios/audio_9710.wav',
 'text': "Salinger said the work President Kennedy , advisers , and members of his staff were doing on the address involved composition and wording , rather than last minute decisions on administration plans to meet the latest Berlin crisis precipitated by Russia's demands and proposals for the city ."}

In [63]:
# part_id = 4
# _id = 19729
# start_index = localized_meta_data[f'part{part_id}'][0]['id']
# final_index = _id - start_index 
# localized_meta_data[f'part{part_id}'][final_index]

{'id': 19729,
 'file_path': 'dataset/part4/audios/audio_19729.wav',
 'text': "-- James P. Mitchell and Sen. Walter H. Jones R-Bergen , last night disagreed on the value of using as a campaign issue a remark by Richard J. Hughes , Democratic gubernatorial candidate , that the GOP is `` Campaigning on the carcass of Eisenhower Republicanism '' ."}

In [59]:
# def replace_bad_sentences(ids, start_index_bad_sentences=0):
#     c = start_index_bad_sentences
#     for _id, part_id in tqdm(ids):
#         start_index = localized_meta_data[f'part{part_id}'][0]['id']
#         final_index = _id - start_index 
#         path = localized_meta_data[f'part{part_id}'][final_index]['file_path']
#         replaced_sentence =  ' '.join(unaccepted_sentences[c])
#         is_ok = tts2(replaced_sentence, rate=22050, write_to_file=True, path=path, return_auido=False, threshold=18)
#         if is_ok:
#             localized_meta_data[f'part{part_id}'][final_index]['text'] = replaced_sentence
#             global_meta_data[_id - 1]['text'] = replaced_sentence
#         else:
#             print((_id, part_id))
#         c += 1
#     print(f'c is {c}')

In [56]:
# replace_bad_sentences(bad_ids, 0)

 88%|████████▊ | 7/8 [01:10<00:10, 10.40s/it]

not accepted because length is [ 0.  0.  0. ...  0. -1. -1.] for path dataset/part5/audios/audio_27377.wav
(27377, 5)


100%|██████████| 8/8 [01:22<00:00, 10.26s/it]

not accepted because length is [ 0.  0.  0. ...  0.  0. -1.] for path dataset/part6/audios/audio_27679.wav
(27679, 6)


In [61]:
# bad_ids = [(27377, 5)]
# replace_bad_sentences(bad_ids, 11)

100%|██████████| 1/1 [00:09<00:00,  9.10s/it]

c is 12


In [64]:
# with open(f'{BASIC_PATH}/global_metadata.json', 'w') as f:
#     json.dump(global_meta_data, f)
# META_DATA_LOCALIZED_PATH = f'{BASIC_PATH}/localized_metadata.json'
# with open(META_DATA_LOCALIZED_PATH, 'w') as f:
#     json.dump(localized_meta_data, f)

In [ ]:
# start_index = localized_meta_data[f'part{part_id}'][0]['id']
# final_index = 1069 + 8641 - start_index 

# print(localized_meta_data[f'part{part_id}'][final_index])
# _id = localized_meta_data[f'part{part_id}'][final_index]['id']
# Audio(filename=f'{BASIC_PATH}/part{part_id}/audios/audio_{_id}.wav')

In [66]:
# shutil.make_archive('dataset_part2', 'zip', f'{BASIC_PATH}/part2')

'/kaggle/working/dataset_part2.zip'

In [67]:
# FileLink(r'dataset_part2.zip')

/kaggle/working/dataset_part2.zip

In [68]:
# shutil.make_archive('dataset_part4', 'zip', f'{BASIC_PATH}/part4')

'/kaggle/working/dataset_part4.zip'

In [69]:
# FileLink(r'dataset_part4.zip')

/kaggle/working/dataset_part4.zip

In [70]:
# os.remove('dataset_part2.zip')

In [76]:
# os.remove('dataset_part6.zip')

In [72]:
# shutil.make_archive('dataset_part5', 'zip', f'{BASIC_PATH}/part5')

'/kaggle/working/dataset_part5.zip'

In [73]:
# FileLink(r'dataset_part5.zip')

/kaggle/working/dataset_part5.zip

In [77]:
# shutil.make_archive('dataset_part6', 'zip', f'{BASIC_PATH}/part6')

'/kaggle/working/dataset_part6.zip'

In [78]:
# FileLink(r'dataset_part6.zip')

/kaggle/working/dataset_part6.zip

In [80]:
# shutil.rmtree('dataset/part2')
# shutil.rmtree('dataset/part4')
# shutil.rmtree('dataset/part5')

In [ ]:
# create_audios(3, META_DATA_LOCALIZED_PATH)

In [81]:
# os.remove('dataset_part6.zip')

In [ ]:
# shutil.make_archive('dataset_part3', 'zip', f'{BASIC_PATH}/part3')

In [ ]:
# FileLink(r'dataset_part3.zip')

In [ ]:
# part_id = 5
# start_index = localized_meta_data[f'part{part_id}'][0]['id']
# final_index = 22080 - start_index 

# print(localized_meta_data[f'part{part_id}'][final_index])
# _id = localized_meta_data[f'part{part_id}'][final_index]['id']
# Audio(filename=f'{BASIC_PATH}/part{part_id}/audios/audio_{_id}.wav')

In [ ]:
# create_audios(4, META_DATA_LOCALIZED_PATH, 20310)

In [ ]:
# os.remove('dataset_part3.zip')

In [ ]:
# create_audios(5, META_DATA_LOCALIZED_PATH, 27378)

In [ ]:
# create_audios(6, META_DATA_LOCALIZED_PATH, 27680)

In [65]:
# shutil.rmtree('dataset/part3')

In [29]:
# create_audios(7, META_DATA_LOCALIZED_PATH, 35321)

100%|██████████| 3320/3320 [2:46:00<00:00,  3.00s/it]  


In [31]:
# create_audios(8, META_DATA_LOCALIZED_PATH)

100%|██████████| 5520/5520 [3:45:20<00:00,  2.45s/it]  


In [32]:
# create_audios(9, META_DATA_LOCALIZED_PATH)

100%|██████████| 5520/5520 [3:49:01<00:00,  2.49s/it]  


In [14]:
create_audios(10, META_DATA_LOCALIZED_PATH, 51165)

100%|██████████| 4009/4009 [2:53:41<00:00,  2.60s/it]  


In [15]:
shutil.make_archive('dataset_part7', 'zip', f'{BASIC_PATH}/part7')

'/kaggle/working/dataset_part7.zip'

In [16]:
FileLink(r'dataset_part7.zip')

/kaggle/working/dataset_part7.zip

In [17]:
shutil.make_archive('dataset_part8', 'zip', f'{BASIC_PATH}/part8')

'/kaggle/working/dataset_part8.zip'

In [18]:
FileLink(r'dataset_part8.zip')

/kaggle/working/dataset_part8.zip

In [21]:
os.remove('dataset_part8.zip')
os.remove('dataset_part7.zip')

In [22]:
shutil.make_archive('dataset_part9', 'zip', f'{BASIC_PATH}/part9')
FileLink(r'dataset_part9.zip')

/kaggle/working/dataset_part9.zip

In [23]:
shutil.make_archive('dataset_part10', 'zip', f'{BASIC_PATH}/part10')
FileLink(r'dataset_part10.zip')

/kaggle/working/dataset_part10.zip

In [24]:
os.remove('dataset_part9.zip')
os.remove('dataset_part10.zip')

In [ ]:

# for extract
# import zipfile
# with zipfile.ZipFile('example.zip', 'r') as zip_ref:
#     zip_ref.extractall('target_dir')

In [ ]:
# def create_audios_old(sentences, start_index, part_id):
#     meta_data = []
#     os.makedirs(f'{BASIC_PATH}/part{part_id}', exist_ok=True)
#     BASIC_PATH_PART = f'{BASIC_PATH}/part{part_id}/audios'
#     os.makedirs(BASIC_PATH_PART, exist_ok=True)
#     META_DATA_PATH = f'{BASIC_PATH}/part{part_id}/meta_data.json'
    
#     for i in tqdm(range(len(sentences))):
#         sentence = ' '.join(sentences[i])
#         path = f'{BASIC_PATH_PART}/audio_{start_index + i}.wav'
#         tts(sentence, rate=22050, write_to_file=True, path=path, return_auido=False)
#         meta_data.append({'id':start_index + i,'file_path': path, 'text': sentence})
        
#         if i % 20 == 0:
#             gc.collect()
#             torch.cuda.empty_cache()
#             torch.cuda.ipc_collect()
            
#     with open(META_DATA_PATH, 'w') as f:
#         json.dump(meta_data, f)

In [ ]:
# create_audios(sublists[0], 1, 1)

In [20]:
localized_meta_data[f'part9'][0]['id']

44161

In [19]:
part_id = 8
a = os.listdir(f'{BASIC_PATH}/part{part_id}/audios')
largest_number = max(int(filename.split('_')[1].split('.')[0]) for filename in a)

print(largest_number)

44160


In [13]:
# META_DATA_LOCALIZED_PATH = f'{BASIC_PATH}/localized_metadata.json'
# with open(META_DATA_LOCALIZED_PATH, 'r') as f:
#     localized_meta_data = json.load(f)
    
start_index = localized_meta_data[f'part{part_id}'][0]['id']

print(localized_meta_data[f'part{part_id}'][largest_number - start_index])
Audio(filename=f'{BASIC_PATH}/part{part_id}/audios/audio_{largest_number}.wav')

{'id': 51164, 'file_path': 'dataset/part10/audios/audio_51164.wav', 'text': 'The search began , in all the rooms , running upstairs , down , opening closets , talking , exclaiming in rushes and gasps.'}


In [ ]:
# meta_data = []
# for i in tqdm(range(len(sentences))):
#     sentence = ' '.join(sentences[i])
#     path = f'{BASIC_PATH}/audio_{i + 1}.wav'
#     _ = tts(sentence, rate=22050, write_to_file='yes', path=path, return_auido=False)
#     meta_data.append({'id':i+1,'file_path': path, 'text': sentence})

In [ ]:
# with open(file_path, 'w') as f:
#     json.dump(my_dict, f)